In [ ]:
import requests  # handles the HTTP requests to JIRA's REST API
import re  # used for regular expression operation
import pandas as pd  # data manipulation
from requests.auth import HTTPBasicAuth  # auth module for secure API requests
from datetime import datetime # used for date calc post-processing

# API req credentials
# HTTPBasicAuth is used to encode the auth sent
# ensures (to the API) that the request is from an authorized user
user = 'intentionally_not_here'  
api_token = 'also_intentionally_not_here'
auth = HTTPBasicAuth(user, api_token)

# jql: string defines the query for JIRA's API using JQL
# fields: the specific fields that JIRA will return for each issue
jql = 'project=your_project'
fields = 'summary,etc.'

# endpoint - this URL is the entry point to JIRA's REST API for searching issues based on the JQL defined earlier
url = "https://yourcompany.atlassian.net/rest/api/3/search"

# function uses a regular expression to search for a tier number within the issue summary text
# the re.IGNORECASE flag makes the search case-insensitive
def extract_tier(summary):
    match = re.search(r'Tier[^0-9]*?(\d+)', summary, re.IGNORECASE)
    return match.group(1) if match else None

# function converts a duration in seconds into a more readable format (days, hours, minutes)
# uses integer division and modulo operations to break down the total seconds into components
def format_time_spent(seconds):
    """formats time spent from seconds into a string of days / hrs / minutes"""
    
    # convert string to integer, in case it's passed as a string
    seconds = int(seconds)
    
    # time periods
    minute = 60
    hour = 60 * minute
    day = 24 * hour

    if seconds < hour:
        return f"{seconds // minute}m"
    elif seconds < day:
        return f"{seconds // hour}h {seconds % hour // minute}m"
    else:
        days_spent = seconds // day
        hours_spent = (seconds % day) // hour
        return f"{days_spent}d {hours_spent}h"

# fetch all issues from JIRA API
# makes paginated requests (requests in chunks) until all issues matching the JQL are retrieved
# uses a loop to handle pagination, adjusting the start_at parameter each time to fetch the next page of results
def fetch_issues(url, auth, jql, fields):
    issues = []
    start_at = 0
    max_results = 100
    total = None
    with requests.Session() as session:
        session.auth = auth
        while total is None or len(issues) < total:
            params = {
                'jql': jql,
                'fields': fields,
                'maxResults': max_results,
                'startAt': start_at,
                'expand': 'changelog'  # includes changelog in the response for history
            }
            response = session.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                total = data['total']
                issues.extend(data['issues'])
                start_at += len(data['issues'])
                print(f"fetched {len(data['issues'])} issues, total so far: {len(issues)} out of {total}")
            else:
                print(f"failed to fetch data: {response.status_code} - {response.reason}")
                break
    return issues

# function fetches all worklogs for a specific issue, accounting for the possibility that worklogs may also be paginated
def fetch_all_worklogs_for_issue(session, issue_key):
    worklog_url = f"https://anfcorp.atlassian.net/rest/api/3/issue/{issue_key}/worklog"
    all_worklogs = []
    start_at = 0
    while True:
        response = session.get(worklog_url, params={'startAt': start_at})
        if response.status_code == 200:
            data = response.json()
            all_worklogs.extend(data['worklogs'])
            if start_at + len(data['worklogs']) < data['total']:
                start_at += len(data['worklogs'])
            else:
                break
        else:
            print(f"failed to fetch worklogs: {response.status_code} - {response.reason}")
            break
    return all_worklogs

# this function iterates over each issue retrieved by fetch_issues to:
# extract specific fields from the issue
# fetch all worklogs using fetch_all_worklogs_for_issue
# process the changelog to determine incremental updates to the time spent on each issue
# the time spent is calculated based on the difference in seconds from the changelog entries, only recording changes (incremental time) rather than total time spent
def extract_worklogs(issues):
    data_rows = []
    with requests.Session() as session:  # reuses the session for multiple requests
        session.auth = auth
        for issue in issues:
            issue_key = issue['key']
            issue_fields = issue['fields']
            tier = extract_tier(issue_fields['summary'])
            components = [comp['name'] for comp in issue_fields.get('components', [])]
            issue_data = {
                'Key': issue_key,
                'Summary': issue_fields['summary'],
                'Issue_Type': issue_fields['issuetype']['name'],
                'Status': issue_fields['status']['name'],
                'Creator': issue_fields['creator']['displayName'],
                'Assignee': issue_fields['assignee']['displayName'] if issue_fields.get('assignee') else 'Unassigned',
                'Reporter': issue_fields['reporter']['displayName'] if issue_fields.get('reporter') else 'Unknown',
                'Created': issue_fields['created'],
                'Updated': issue_fields['updated'],
                'Labels': ', '.join(issue_fields['labels']),
                'Components': ', '.join(components),
                'Tier': tier
            }
            
            # fetch all worklogs if they are paginated
            all_worklogs = fetch_all_worklogs_for_issue(session, issue_key)
            if not all_worklogs:
                data_rows.append(issue_data)
            for log in all_worklogs:
                time_spent_formatted = format_time_spent(log['timeSpentSeconds'])
                log_data = issue_data.copy()
                log_data.update({
                    'Worker': log['author']['displayName'],
                    'Time_Spent': time_spent_formatted,
                    'Worklog_Created': log['started'],
                    'Log_Type': 'Work Log'
                })
                data_rows.append(log_data)

            # processes the changelog to find updates to time spent
            for history in issue.get('changelog', {}).get('histories', []):
                for item in history['items']:
                    if item['field'] == 'timespent' or item['field'] == 'Time Spent':
                        
                        # converts changelog timespent from and to strings to seconds
                        from_seconds = int(item['fromString'] or 0)  # if 'fromString' is None, default to 0
                        to_seconds = int(item['toString'] or 0)  # if 'toString' is None, default to 0
                        
                        # calc incremental time spent
                        timespent_change_seconds = to_seconds - from_seconds
                        
                        # skips if no change in time spent
                        if timespent_change_seconds <= 0:
                            continue
                        
                        # formatting the incremental time spent
                        time_spent_formatted = format_time_spent(timespent_change_seconds)
                        time_spent_entry = issue_data.copy()
                        time_spent_entry.update({
                            'Worker': history['author']['displayName'],
                            'Time_Spent': time_spent_formatted,
                            'Worklog_Created': history['created'],
                            'Log_Type': 'Time Spent Update'
                        })
                        data_rows.append(time_spent_entry)
    return data_rows

# calls fetch_issues to get all issues and then processes each issue with extract_worklogs to extract worklogs and time spent updates
issues = fetch_issues(url, auth, jql, fields)
data_rows = extract_worklogs(issues)

# adds a 'Worklog_Created_Date' key only if 'Worklog_Created' is present
for row in data_rows:
    if 'Worklog_Created' in row:
        # extracts the date part from the 'Worklog_Created' timestamp
        # uses datetime.strptime to parse the timestamp into a datetime object and then takes the date part
        row['Worklog_Created_Date'] = datetime.strptime(row['Worklog_Created'], "%Y-%m-%dT%H:%M:%S.%f%z").date()

# identifies the "Time Spent Update" entries to remove
rows_to_remove = set()
for work_log in filter(lambda d: d.get('Log_Type') == 'Work Log' and 'Worklog_Created' in d, data_rows):
    for time_spent_update in filter(lambda d: d.get('Log_Type') == 'Time Spent Update' and 'Worklog_Created' in d, data_rows):
        
        # nested loop checks each 'Work Log' against each 'Time Spent Update'
        # uses the get method to access 'Log_Type' and ensures 'Worklog_Created' exists
        # if they are by the same worker on the same date, it marks the 'Time Spent Update' for removal (work logs are prioritized)
        if (work_log['Worker'] == time_spent_update['Worker'] and
            work_log['Worklog_Created_Date'] == time_spent_update['Worklog_Created_Date']):
            rows_to_remove.add(time_spent_update['Worklog_Created'])

# retains all original rows, removing only the duplicates identified
final_data_rows = []
for row in data_rows:

    # rows that are 'Work Log' type are always kept
    # 'Time Spent Update' rows are kept only if they're not marked for removal
    # if no 'Log_Type' or 'Worklog_Created' keys -> keep the row
    if 'Log_Type' not in row or 'Worklog_Created' not in row:
        final_data_rows.append(row)
    # if 'Log_Type' is 'Work Log' -> keep the row
    elif row['Log_Type'] == 'Work Log':
        final_data_rows.append(row)
    # if 'Log_Type' is 'Time Spent Update' + it's not in the rows to remove -> keep the row
    elif row['Log_Type'] == 'Time Spent Update' and row['Worklog_Created'] not in rows_to_remove:
        final_data_rows.append(row)

# pandas df from the processed data and writes it to a csv file
# index=False is used so that pandas doesn't write the df index as a separate column in the csv output
df = pd.DataFrame(final_data_rows)

# path to fabric lakehouse
out_path = "/lakehouse/default/Files/your_file_name.csv"

# csv
df.to_csv(out_path, index=False)